# Robustness Analysis: Adjoint and Toggle-Frame Objectives Subject to Either Multiplicative or Additive Errors

This notebook compares the performance of the adjoint and toggling-frame robustness objectives for multiplicative and additive error terms in the system's Hamiltonian. 

## Imports

In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
using PiccoloQuantumObjects
using QuantumCollocation
using ForwardDiff
using LinearAlgebra
using Plots
using SparseArrays
using NamedTrajectories
using Statistics
using CairoMakie
using Random
using ColorSchemes
using Makie
using Printf

In [ ]:
# Problem parameters
T = 25
Δt = 0.2
U_goal = GATES.H
H_drive = [PAULIS.X, PAULIS.Y, PAULIS.Z]
piccolo_opts = PiccoloOptions(verbose=false)
pretty_print(X::AbstractMatrix) = Base.show(stdout, "text/plain", X);
sys = QuantumSystem(H_drive)
# Adjoint
∂ₑHₐ = [PAULIS.X, PAULIS.Y, PAULIS.Z]
varsys = VariationalQuantumSystem(
    H_drive,
    ∂ₑHₐ
)


Setup metrics

In [ ]:
function SpaceCurve(traj::NamedTrajectory, U_goal::AbstractMatrix{<:Number}, H_err::AbstractMatrix{<:Number})
    T = traj.T
    first_order_terms = Vector{Matrix{ComplexF64}}(undef, T)
    first_order_integral = zeros(ComplexF64, size(U_goal))

    for i in 1:T
        U = iso_vec_to_operator(traj.Ũ⃗[:, i])
        first_order_integral += U' * H_err * U
        first_order_terms[i] = first_order_integral
    end
    d = size(U_goal)[1]
    space_curve = [[real(tr(PAULIS.X * first_order_terms[t] / (d * T))),
                    real(tr(PAULIS.Y * first_order_terms[t] / (d * T))),
                    real(tr(PAULIS.Z * first_order_terms[t] / (d * T)))] for t in 1:T] 
    return space_curve
end

function space_curve_robustness(traj::NamedTrajectory, U_goal::AbstractMatrix, H_err::AbstractMatrix)
    curve = SpaceCurve(traj, U_goal, H_err)
    # Use the norm of the final point as a robustness measure
    # (larger values indicate more accumulated error sensitivity)
    final_point = curve[end]
    return norm(final_point)
end

function width_robustness(system::AbstractQuantumSystem, traj::NamedTrajectory; thresh::Float64=0.999)
    F = 1.0
    drift = system.H.H_drift
    drive = system.H.H_drives
    pauls = [PAULIS.X, PAULIS.Y, PAULIS.Z]
    widths = []
    for i in 1:3
        ε = 0.0
        err = pauls[i]
        F = 1.0
        while (ε < 0.5 && F >= thresh)
            noisy_drift = drift + ε * err
            noisy_sys = QuantumSystem(noisy_drift, drive)
            F = unitary_rollout_fidelity(traj, noisy_sys)
            ε += 0.0001
        end
        push!(widths, ε)
    end
    return widths
end
# Define the variational objective function
function var_obj(traj::NamedTrajectory, H_drive::Vector{Matrix{ComplexF64}}, error_op::Matrix{ComplexF64})
    varsys = VariationalQuantumSystem(
        H_drive,
        [error_op]
    )
    T = traj.T
    Δt = traj.Δt[1]
    ww = iso_vec_to_operator(variational_unitary_rollout(traj, varsys)[2][1][:,end])
    d = size(ww)[1]
    return norm(tr(ww'ww)) / (T * Δt)^2 / d
end


In [ ]:
# Define parameter ranges (logarithmic scale)
a_bounds = 10 .^ range(-.5, 0.5, length=10)  # 1e0 to 1e1
dda_bounds = 10 .^ range(-0.5, 0.5, length=25)  # 1e-1 to 1e1

n_seeds = 2  # Number of random seeds to test
n_drives = sys.n_drives
var_n_drives = varsys.n_drives
variational_scales = fill(1.0, length(varsys.G_vars))

println("Parameter ranges:")
println("a_bounds: ", a_bounds)
println("dda_bounds: ", dda_bounds)
println("Number of seeds: ", n_seeds)

# Initialize results storage - now with an extra dimension for seeds
trajectories = Array{Any}(undef, length(a_bounds), length(dda_bounds), n_seeds)
fidelities = zeros(length(a_bounds), length(dda_bounds), n_seeds)
convergence_times = zeros(length(a_bounds), length(dda_bounds), n_seeds)
final_costs = zeros(length(a_bounds), length(dda_bounds), n_seeds)

# Define seeds to use consistently across all parameter combinations
seed_values = [42, 123]

# Parameter sweep
println("\nStarting parameter sweep with multiple seeds...")
for (i, a_bound) in enumerate(a_bounds)
    for (j, dda_bound) in enumerate(dda_bounds)
        println("\n" * "="^60)
        println("Parameter set: a_bound=$(Printf.@sprintf("%.3e", a_bound)), dda_bound=$(Printf.@sprintf("%.3e", dda_bound))")
        println("="^60)
        
        for (seed_idx, seed) in enumerate(seed_values)
            println("\n  Seed $seed_idx (value=$seed):")
            
            # Set random seed for reproducibility
            Random.seed!(seed)
            
            # Create variational problem with current parameters
            var_prob_test = UnitaryVariationalProblem(
                varsys, U_goal, T, Δt;
                robust_times=[[T], [T], [T]],
                a_bound = a_bound,
                dda_bound = dda_bound,
                piccolo_options=piccolo_opts
            )
            
            # Solve with timing
            elapsed_time = @elapsed begin
                solve!(var_prob_test, max_iter=750, print_level=0, 
                        options=IpoptOptions(eval_hessian=false))
                solve!(var_prob_test, max_iter=30, print_level=0)
            end
            
            # Calculate final fidelity
            fidelity = unitary_rollout_fidelity(var_prob_test.trajectory, sys)
            
            # Store results
            trajectories[i, j, seed_idx] = var_prob_test.trajectory
            fidelities[i, j, seed_idx] = fidelity
            convergence_times[i, j, seed_idx] = elapsed_time
            Z_vec = vec(var_prob_test.trajectory)
            final_costs[i, j, seed_idx] = var_prob_test.objective.L(Z_vec)
            
            println("    -> Fidelity: $(Printf.@sprintf("%.6f", fidelity)), Time: $(Printf.@sprintf("%.2f", elapsed_time))s, Cost: $(Printf.@sprintf("%.4e", final_costs[i, j, seed_idx]))")
        end
        
        # Compare seeds for this parameter combination
        fid_diff = abs(fidelities[i, j, 1] - fidelities[i, j, 2])
        println("\n  Seed comparison:")
        println("    Fidelity difference: $(Printf.@sprintf("%.6f", fid_diff))")
        println("    Best seed: $(fidelities[i, j, 1] >= fidelities[i, j, 2] ? "Seed 1" : "Seed 2")")
    end
end

# Compute statistics across seeds
mean_fidelities = mean(fidelities, dims=3)[:, :, 1]
std_fidelities = std(fidelities, dims=3)[:, :, 1]
max_fidelities = maximum(fidelities, dims=3)[:, :, 1]
min_fidelities = minimum(fidelities, dims=3)[:, :, 1]

mean_times = mean(convergence_times, dims=3)[:, :, 1]
mean_costs = mean(final_costs, dims=3)[:, :, 1]

println("\n" * "="^80)
println("MULTI-SEED ANALYSIS COMPLETE")
println("="^80)
println("\nFidelity statistics across seeds:")
println("  Mean of means: $(Printf.@sprintf("%.6f", mean(mean_fidelities)))")
println("  Mean of stds: $(Printf.@sprintf("%.6f", mean(std_fidelities)))")
println("  Max std: $(Printf.@sprintf("%.6f", maximum(std_fidelities)))")
println("  Overall best: $(Printf.@sprintf("%.6f", maximum(fidelities)))")
println("  Overall worst: $(Printf.@sprintf("%.6f", minimum(fidelities)))")

In [ ]:
# Create comprehensive visualization with multi-seed analysis
f = Figure(size=(1800, 1200))

# 1. Mean Fidelity 3D Surface Plot
ax3d = Axis3(f[1, 1:2],
    title="Mean Fidelity Across Seeds vs a_bound and dda_bound",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)",
    zlabel="Mean Fidelity",
    aspect=(1, 1, 0.5)
)

# Create meshgrid for 3D plot
x_3d = repeat(log10.(a_bounds), 1, length(dda_bounds))
y_3d = repeat(log10.(dda_bounds)', length(a_bounds), 1)
z_3d = mean_fidelities

# Create 3D surface
Makie.surface!(ax3d, x_3d, y_3d, z_3d, colormap=:viridis, alpha=0.8)
Makie.contour3d!(ax3d, x_3d, y_3d, z_3d, levels=10, linewidth=1, alpha=0.6)

# 2. Mean Fidelity 2D Heatmap
ax_heat_mean = Axis(f[2, 1],
    title="Mean Fidelity Heatmap",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)

hm_mean = Makie.heatmap!(ax_heat_mean, log10.(a_bounds), log10.(dda_bounds), mean_fidelities,
             colormap=:viridis, colorrange=(0.95, 1.0))
Colorbar(f[2, 2], hm_mean, label="Mean Fidelity")

Makie.contour!(ax_heat_mean, log10.(a_bounds), log10.(dda_bounds), mean_fidelities,
        levels=[0.99, 0.995, 0.999], color=:white, linewidth=1.5)

# 3. Standard Deviation Heatmap (shows seed variability)
ax_heat_std = Axis(f[1, 3],
    title="Fidelity Std Dev Across Seeds",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)

hm_std = Makie.heatmap!(ax_heat_std, log10.(a_bounds), log10.(dda_bounds), std_fidelities,
             colormap=:reds)
Colorbar(f[1, 4], hm_std, label="Std Dev")

# 4. Max-Min Difference (shows range across seeds)
fidelity_range = max_fidelities - min_fidelities
ax_heat_range = Axis(f[2, 3],
    title="Fidelity Range Across Seeds",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)

hm_range = Makie.heatmap!(ax_heat_range, log10.(a_bounds), log10.(dda_bounds), fidelity_range,
             colormap=:plasma)
Colorbar(f[2, 4], hm_range, label="Max - Min")

# 5. Cross-section comparison showing variability
ax_cross1 = Axis(f[3, 1:2],
    title="Mean Fidelity vs a_bound with Std Dev Bands",
    xlabel="log₁₀(a_bound)",
    ylabel="Fidelity"
)

# Select middle dda_bound for cross-section
mid_dda_idx = round(Int, length(dda_bounds)/2)

# Plot mean with error bands
lines!(ax_cross1, log10.(a_bounds), mean_fidelities[:, mid_dda_idx],
       label="Mean", color=:blue, linewidth=3)
band!(ax_cross1, log10.(a_bounds),
      mean_fidelities[:, mid_dda_idx] .- std_fidelities[:, mid_dda_idx],
      mean_fidelities[:, mid_dda_idx] .+ std_fidelities[:, mid_dda_idx],
      color=(:blue, 0.3), label="±1 Std Dev")

# Plot individual seeds
for seed_idx in 1:n_seeds
    lines!(ax_cross1, log10.(a_bounds), fidelities[:, mid_dda_idx, seed_idx],
           label="Seed $seed_idx", linewidth=1.5, linestyle=:dash)
end

axislegend(ax_cross1, position=:rb)

# 6. Cross-section for dda_bound
ax_cross2 = Axis(f[3, 3:4],
    title="Mean Fidelity vs dda_bound with Std Dev Bands",
    xlabel="log₁₀(dda_bound)",
    ylabel="Fidelity"
)

# Select middle a_bound for cross-section
mid_a_idx = round(Int, length(a_bounds)/2)

lines!(ax_cross2, log10.(dda_bounds), mean_fidelities[mid_a_idx, :],
       label="Mean", color=:red, linewidth=3)
band!(ax_cross2, log10.(dda_bounds),
      mean_fidelities[mid_a_idx, :] .- std_fidelities[mid_a_idx, :],
      mean_fidelities[mid_a_idx, :] .+ std_fidelities[mid_a_idx, :],
      color=(:red, 0.3), label="±1 Std Dev")

# Plot individual seeds
for seed_idx in 1:n_seeds
    lines!(ax_cross2, log10.(dda_bounds), fidelities[mid_a_idx, :, seed_idx],
           label="Seed $seed_idx", linewidth=1.5, linestyle=:dash)
end

axislegend(ax_cross2, position=:rb)

# Add overall title
Label(f[0, 1:4], "Multi-Seed Variational Quantum Control: Parameter Sensitivity Analysis",
      fontsize=20, font="bold")

# Display the figure
display(f)

# Print summary statistics
println("\n" * "="^60)
println("SUMMARY STATISTICS (MULTI-SEED)")
println("="^60)

max_mean_fidelity = maximum(mean_fidelities)
max_idx = argmax(mean_fidelities)
best_a = a_bounds[max_idx[1]]
best_dda = dda_bounds[max_idx[2]]

println("\nBest mean fidelity: $(Printf.@sprintf("%.6f", max_mean_fidelity))")
println("Best parameters: a_bound = $(Printf.@sprintf("%.3e", best_a)), dda_bound = $(Printf.@sprintf("%.3e", best_dda))")
println("Std dev at best params: $(Printf.@sprintf("%.6f", std_fidelities[max_idx]))")
println("Range at best params: $(Printf.@sprintf("%.6f", fidelity_range[max_idx]))")

# Find parameters with both high mean and low variance
high_quality_mask = (mean_fidelities .> 0.999) .& (std_fidelities .< 0.001)
n_high_quality = sum(high_quality_mask)
println("\nParameter combinations with fidelity > 0.999 AND std < 0.001: $n_high_quality / $(length(mean_fidelities))")

# Identify most stable parameter regions (low variance)
min_std = minimum(std_fidelities)
min_std_idx = argmin(std_fidelities)
println("\nMost stable parameters (lowest std dev):")
println("  a_bound = $(Printf.@sprintf("%.3e", a_bounds[min_std_idx[1]])), dda_bound = $(Printf.@sprintf("%.3e", dda_bounds[min_std_idx[2]]))")
println("  Std dev: $(Printf.@sprintf("%.6f", min_std))")
println("  Mean fidelity: $(Printf.@sprintf("%.6f", mean_fidelities[min_std_idx]))")

# Seed sensitivity analysis
avg_std = mean(std_fidelities)
max_std = maximum(std_fidelities)
max_std_idx = argmax(std_fidelities)
println("\nSeed sensitivity across all parameters:")
println("  Average std dev: $(Printf.@sprintf("%.6f", avg_std))")
println("  Maximum std dev: $(Printf.@sprintf("%.6f", max_std))")
println("  Most sensitive at: a_bound = $(Printf.@sprintf("%.3e", a_bounds[max_std_idx[1]])), dda_bound = $(Printf.@sprintf("%.3e", dda_bounds[max_std_idx[2]]))")

In [ ]:
# Calculate robustness metrics for all trajectories (all seeds)
println("\nCalculating robustness metrics for all seeds...")

# Initialize storage for robustness metrics (now with seed dimension)
space_curve_robustness_values = zeros(length(a_bounds), length(dda_bounds), n_seeds)
width_robustness_x = zeros(length(a_bounds), length(dda_bounds), n_seeds)
width_robustness_y = zeros(length(a_bounds), length(dda_bounds), n_seeds)
width_robustness_z = zeros(length(a_bounds), length(dda_bounds), n_seeds)
width_robustness_avg = zeros(length(a_bounds), length(dda_bounds), n_seeds)

# Calculate robustness for each trajectory
for (i, a_bound) in enumerate(a_bounds)
    for (j, dda_bound) in enumerate(dda_bounds)
        for seed_idx in 1:n_seeds
            if !isnothing(trajectories[i, j, seed_idx])
                traj = trajectories[i, j, seed_idx]
                
                # Space curve robustness (average over all Pauli errors)
                Xerr = space_curve_robustness(traj, U_goal, PAULIS.X)
                Yerr = space_curve_robustness(traj, U_goal, PAULIS.Y)
                Zerr = space_curve_robustness(traj, U_goal, PAULIS.Z)
                space_curve_robustness_values[i, j, seed_idx] = 1-(Xerr + Yerr + Zerr) / 3
                
                # Width robustness
                widths = width_robustness(sys, traj, thresh=0.99)
                width_robustness_x[i, j, seed_idx] = widths[1]
                width_robustness_y[i, j, seed_idx] = widths[2]
                width_robustness_z[i, j, seed_idx] = widths[3]
                width_robustness_avg[i, j, seed_idx] = mean(widths)
                
                println("a=$(Printf.@sprintf("%.3e", a_bound)), dda=$(Printf.@sprintf("%.3e", dda_bound)), seed=$seed_idx: " *
                       "SC_robust=$(Printf.@sprintf("%.4f", space_curve_robustness_values[i, j, seed_idx])), " *
                       "Width_avg=$(Printf.@sprintf("%.4f", width_robustness_avg[i, j, seed_idx]))")
            end
        end
    end
end

# Compute statistics across seeds for robustness metrics
mean_sc_robust = mean(space_curve_robustness_values, dims=3)[:, :, 1]
std_sc_robust = std(space_curve_robustness_values, dims=3)[:, :, 1]
mean_width_robust = mean(width_robustness_avg, dims=3)[:, :, 1]
std_width_robust = std(width_robustness_avg, dims=3)[:, :, 1]

# Create comprehensive robustness visualization
f_robust = Figure(size=(2000, 1600))

# Row 1: Mean Space Curve Robustness
ax_sc_3d = Axis3(f_robust[1, 1:2],
    title="Mean Space Curve Robustness (Adjoint)",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)",
    zlabel="SC Robustness",
    aspect=(1, 1, 0.5)
)

x_3d = repeat(log10.(a_bounds), 1, length(dda_bounds))
y_3d = repeat(log10.(dda_bounds)', length(a_bounds), 1)
Makie.surface!(ax_sc_3d, x_3d, y_3d, mean_sc_robust, colormap=:plasma, alpha=0.8)
Makie.contour3d!(ax_sc_3d, x_3d, y_3d, mean_sc_robust, levels=10, linewidth=1, alpha=0.6)

ax_sc_heat = Axis(f_robust[1, 3],
    title="Mean SC Robustness (Adjoint)",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)
hm_sc = Makie.heatmap!(ax_sc_heat, log10.(a_bounds), log10.(dda_bounds), mean_sc_robust, colormap=:plasma)
Colorbar(f_robust[1, 4], hm_sc, label="Mean SC Robust")

ax_sc_std = Axis(f_robust[1, 5],
    title="SC Robustness Std Dev (Adjoint)",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)
hm_sc_std = Makie.heatmap!(ax_sc_std, log10.(a_bounds), log10.(dda_bounds), std_sc_robust, colormap=:reds)
Colorbar(f_robust[1, 6], hm_sc_std, label="Std Dev")

# Row 2: Mean Width Robustness
ax_w_3d = Axis3(f_robust[2, 1:2],
    title="Mean Width Robustness (Adjoint)",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)",
    zlabel="Width Robustness",
    aspect=(1, 1, 0.5)
)

Makie.surface!(ax_w_3d, x_3d, y_3d, mean_width_robust, colormap=:turbo, alpha=0.8)
Makie.contour3d!(ax_w_3d, x_3d, y_3d, mean_width_robust, levels=10, linewidth=1, alpha=0.6)

ax_w_heat = Axis(f_robust[2, 3],
    title="Mean Width Robustness (Adjoint)",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)
hm_w = Makie.heatmap!(ax_w_heat, log10.(a_bounds), log10.(dda_bounds), mean_width_robust, colormap=:turbo)
Colorbar(f_robust[2, 4], hm_w, label="Mean Width")

ax_w_std = Axis(f_robust[2, 5],
    title="Width Robustness Std Dev",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)
hm_w_std = Makie.heatmap!(ax_w_std, log10.(a_bounds), log10.(dda_bounds), std_width_robust, colormap=:reds)
Colorbar(f_robust[2, 6], hm_w_std, label="Std Dev")

# Row 3: Correlation analysis
ax_corr = Axis(f_robust[3, 1:3],
    title="Correlation: Space Curve vs Width Robustness (All Seeds)",
    xlabel="Space Curve Robustness",
    ylabel="Width Robustness"
)

# Flatten all seeds for correlation plot
sc_all = vec(space_curve_robustness_values)
w_all = vec(width_robustness_avg)
fid_all = vec(fidelities)

# Create scatter with seed coloring
colors_seeds = [:red, :blue, :green, :orange, :purple]
for seed_idx in 1:n_seeds
    sc_seed = vec(space_curve_robustness_values[:, :, seed_idx])
    w_seed = vec(width_robustness_avg[:, :, seed_idx])
    Makie.scatter!(ax_corr, sc_seed, w_seed,
              label="Seed $seed_idx",
              color=colors_seeds[seed_idx],
              markersize=6, alpha=0.6)
end

correlation = cor(sc_all, w_all)
text!(ax_corr, 0.05, 0.95, text="Overall Correlation: $(Printf.@sprintf("%.3f", correlation))",
      space=:relative, fontsize=14, font="bold")
axislegend(ax_corr, position=:rb)

# Row 3: Summary statistics
ax_summary = Axis(f_robust[3, 4:6],
    title="Multi-Seed Robustness Summary"
)
hidedecorations!(ax_summary)
hidespines!(ax_summary)

summary_text = """
Space Curve Robustness:
  Mean across all: $(Printf.@sprintf("%.4f", mean(mean_sc_robust)))
  Std across params: $(Printf.@sprintf("%.4f", std(mean_sc_robust)))
  Avg std across seeds: $(Printf.@sprintf("%.4f", mean(std_sc_robust)))
  Min mean: $(Printf.@sprintf("%.4f", minimum(mean_sc_robust)))
  Max mean: $(Printf.@sprintf("%.4f", maximum(mean_sc_robust)))

Width Robustness:
  Mean across all: $(Printf.@sprintf("%.4f", mean(mean_width_robust)))
  Std across params: $(Printf.@sprintf("%.4f", std(mean_width_robust)))
  Avg std across seeds: $(Printf.@sprintf("%.4f", mean(std_width_robust)))
  Min mean: $(Printf.@sprintf("%.4f", minimum(mean_width_robust)))
  Max mean: $(Printf.@sprintf("%.4f", maximum(mean_width_robust)))

Seed Variability:
  Params with high SC std (>0.01): $(sum(std_sc_robust .> 0.01))
  Params with high W std (>0.01): $(sum(std_width_robust .> 0.01))

Best Combined (Fidelity + Robustness):
  Using mean fidelity > 0.999 & mean width > 0.1
  Count: $(sum((mean_fidelities .> 0.999) .& (mean_width_robust .> 0.1)))
"""

text!(ax_summary, 0.05, 0.5, text=summary_text,
      space=:relative, fontsize=11, font="monospace")

# Row 4: Cross-sections with seed bands
mid_dda = round(Int, length(dda_bounds)/2)
mid_a = round(Int, length(a_bounds)/2)

ax_cross_sc = Axis(f_robust[4, 1:3],
    title="Space Curve Robustness vs a_bound (with seed variability)",
    xlabel="log₁₀(a_bound)",
    ylabel="SC Robustness"
)

lines!(ax_cross_sc, log10.(a_bounds), mean_sc_robust[:, mid_dda],
       label="Mean", color=:blue, linewidth=3)
band!(ax_cross_sc, log10.(a_bounds),
      mean_sc_robust[:, mid_dda] .- std_sc_robust[:, mid_dda],
      mean_sc_robust[:, mid_dda] .+ std_sc_robust[:, mid_dda],
      color=(:blue, 0.3), label="±1 Std")

for seed_idx in 1:n_seeds
    lines!(ax_cross_sc, log10.(a_bounds), space_curve_robustness_values[:, mid_dda, seed_idx],
           label="Seed $seed_idx", linewidth=1.5, linestyle=:dash, color=colors_seeds[seed_idx])
end
axislegend(ax_cross_sc, position=:rt)

ax_cross_w = Axis(f_robust[4, 4:6],
    title="Width Robustness vs dda_bound (with seed variability)",
    xlabel="log₁₀(dda_bound)",
    ylabel="Width Robustness"
)

lines!(ax_cross_w, log10.(dda_bounds), mean_width_robust[mid_a, :],
       label="Mean", color=:red, linewidth=3)
band!(ax_cross_w, log10.(dda_bounds),
      mean_width_robust[mid_a, :] .- std_width_robust[mid_a, :],
      mean_width_robust[mid_a, :] .+ std_width_robust[mid_a, :],
      color=(:red, 0.3), label="±1 Std")

for seed_idx in 1:n_seeds
    lines!(ax_cross_w, log10.(dda_bounds), width_robustness_avg[mid_a, :, seed_idx],
           label="Seed $seed_idx", linewidth=1.5, linestyle=:dash, color=colors_seeds[seed_idx])
end
axislegend(ax_cross_w, position=:lt)

Label(f_robust[0, :], "Multi-Seed Robustness Analysis: Parameter Sensitivity",
      fontsize=20, font="bold")

display(f_robust)

println("\nMulti-seed robustness analysis complete!")

In [7]:
using Dates
using JLD2

# Create a descriptive folder name with timestamp
timestamp = Dates.format(now(), "yyyymmdd_HHMMSS")
sweep_params = "a$(length(a_bounds))_dda$(length(dda_bounds))_seeds$(n_seeds)"
folder_name = "param_sweep_$(sweep_params)_$(timestamp)"
save_path = joinpath("artifacts", folder_name)

# Create the directory structure
mkpath(save_path)

println("\nSaving results to: $save_path")

# Save the trajectories and all results
@save joinpath(save_path, "results.jld2") trajectories fidelities convergence_times final_costs a_bounds dda_bounds seed_values

# Save summary statistics
@save joinpath(save_path, "statistics.jld2") mean_fidelities std_fidelities max_fidelities min_fidelities mean_times mean_costs

# Save metadata and configuration
metadata = Dict(
    "timestamp" => timestamp,
    "T" => T,
    "Δt" => Δt,
    "n_seeds" => n_seeds,
    "n_a_points" => length(a_bounds),
    "n_dda_points" => length(dda_bounds),
    "total_runs" => length(a_bounds) * length(dda_bounds) * n_seeds,
    "a_range" => (minimum(a_bounds), maximum(a_bounds)),
    "dda_range" => (minimum(dda_bounds), maximum(dda_bounds)),
    "seed_values" => seed_values,
    "gate" => "Hadamard"
)

@save joinpath(save_path, "metadata.jld2") metadata

# Save a human-readable summary
open(joinpath(save_path, "summary.txt"), "w") do io
    println(io, "Parameter Sweep Results")
    println(io, "="^60)
    println(io, "Timestamp: $timestamp")
    println(io, "\nConfiguration:")
    println(io, "  Gate: Hadamard")
    println(io, "  Time horizon (T): $T")
    println(io, "  Time step (Δt): $Δt")
    println(io, "  Number of seeds: $n_seeds")
    println(io, "  Seed values: $seed_values")
    println(io, "\nParameter Grid:")
    println(io, "  a_bounds: $(length(a_bounds)) points from $(minimum(a_bounds)) to $(maximum(a_bounds))")
    println(io, "  dda_bounds: $(length(dda_bounds)) points from $(minimum(dda_bounds)) to $(maximum(dda_bounds))")
    println(io, "  Total optimization runs: $(length(a_bounds) * length(dda_bounds) * n_seeds)")
    println(io, "\nResults Summary:")
    println(io, "  Mean fidelity: $(Printf.@sprintf("%.6f", mean(mean_fidelities)))")
    println(io, "  Mean std: $(Printf.@sprintf("%.6f", mean(std_fidelities)))")
    println(io, "  Best fidelity: $(Printf.@sprintf("%.6f", maximum(fidelities)))")
    println(io, "  Worst fidelity: $(Printf.@sprintf("%.6f", minimum(fidelities)))")
    println(io, "  Mean convergence time: $(Printf.@sprintf("%.2f", mean(mean_times))) seconds")
end

println("✓ Saved trajectories and results")
println("✓ Saved statistics")
println("✓ Saved metadata")
println("✓ Saved summary.txt")
println("\nAll data saved successfully to: $save_path")

In [9]:
using Dates
using JLD2

# Create a descriptive folder name with timestamp
timestamp = Dates.format(now(), "yyyymmdd_HHMMSS")
sweep_params = "a$(length(a_bounds))_dda$(length(dda_bounds))_seeds$(n_seeds)"
folder_name = "param_sweep_$(sweep_params)_$(timestamp)"
save_path = joinpath("artifacts", folder_name)

# Create the directory structure
mkpath(save_path)
mkpath(joinpath(save_path, "trajectories"))

println("\nSaving results to: $save_path")

# Save individual trajectories with descriptive names
println("Saving individual trajectories...")
for (i, a_bound) in enumerate(a_bounds)
    for (j, dda_bound) in enumerate(dda_bounds)
        for (seed_idx, seed) in enumerate(seed_values)
            traj = trajectories[i, j, seed_idx]
            filename = "traj_a$(i)_dda$(j)_seed$(seed_idx).jld2"
            @save joinpath(save_path, "trajectories", filename) traj
        end
    end
end
println("✓ Saved $(length(a_bounds) * length(dda_bounds) * n_seeds) individual trajectories")

# Save aggregated results
@save joinpath(save_path, "results.jld2") fidelities convergence_times final_costs a_bounds dda_bounds seed_values

# Save summary statistics
@save joinpath(save_path, "statistics.jld2") mean_fidelities std_fidelities max_fidelities min_fidelities mean_times mean_costs

# Save metadata and configuration
metadata = Dict(
    "timestamp" => timestamp,
    "T" => T,
    "Δt" => Δt,
    "n_seeds" => n_seeds,
    "n_a_points" => length(a_bounds),
    "n_dda_points" => length(dda_bounds),
    "total_runs" => length(a_bounds) * length(dda_bounds) * n_seeds,
    "a_range" => (minimum(a_bounds), maximum(a_bounds)),
    "dda_range" => (minimum(dda_bounds), maximum(dda_bounds)),
    "seed_values" => seed_values,
    "gate" => "Hadamard"
)

@save joinpath(save_path, "metadata.jld2") metadata

# Create an index file to help locate trajectories
open(joinpath(save_path, "trajectory_index.txt"), "w") do io
    println(io, "Trajectory Index")
    println(io, "="^80)
    println(io, "Format: traj_a{i}_dda{j}_seed{k}.jld2")
    println(io, "")
    println(io, "Parameters:")
    for (i, a_bound) in enumerate(a_bounds)
        println(io, "  a[$i] = $(Printf.@sprintf("%.6e", a_bound))")
    end
    println(io, "")
    for (j, dda_bound) in enumerate(dda_bounds)
        println(io, "  dda[$j] = $(Printf.@sprintf("%.6e", dda_bound))")
    end
    println(io, "")
    for (seed_idx, seed) in enumerate(seed_values)
        println(io, "  seed[$seed_idx] = $seed")
    end
    println(io, "")
    println(io, "="^80)
    println(io, "Summary by parameter combination:")
    println(io, "")
    
    for (i, a_bound) in enumerate(a_bounds)
        for (j, dda_bound) in enumerate(dda_bounds)
            println(io, "a[$i]=$(Printf.@sprintf("%.3e", a_bound)), dda[$j]=$(Printf.@sprintf("%.3e", dda_bound)):")
            for (seed_idx, seed) in enumerate(seed_values)
                fid = fidelities[i, j, seed_idx]
                time = convergence_times[i, j, seed_idx]
                cost = final_costs[i, j, seed_idx]
                println(io, "  seed$seed_idx: F=$(Printf.@sprintf("%.6f", fid)), t=$(Printf.@sprintf("%.2f", time))s, cost=$(Printf.@sprintf("%.4e", cost)) → traj_a$(i)_dda$(j)_seed$(seed_idx).jld2")
            end
            println(io, "")
        end
    end
end

# Save a human-readable summary
open(joinpath(save_path, "summary.txt"), "w") do io
    println(io, "Parameter Sweep Results")
    println(io, "="^60)
    println(io, "Timestamp: $timestamp")
    println(io, "\nConfiguration:")
    println(io, "  Gate: Hadamard")
    println(io, "  Time horizon (T): $T")
    println(io, "  Time step (Δt): $Δt")
    println(io, "  Number of seeds: $n_seeds")
    println(io, "  Seed values: $seed_values")
    println(io, "\nParameter Grid:")
    println(io, "  a_bounds: $(length(a_bounds)) points from $(minimum(a_bounds)) to $(maximum(a_bounds))")
    println(io, "  dda_bounds: $(length(dda_bounds)) points from $(minimum(dda_bounds)) to $(maximum(dda_bounds))")
    println(io, "  Total optimization runs: $(length(a_bounds) * length(dda_bounds) * n_seeds)")
    println(io, "\nResults Summary:")
    println(io, "  Mean fidelity: $(Printf.@sprintf("%.6f", mean(mean_fidelities)))")
    println(io, "  Mean std: $(Printf.@sprintf("%.6f", mean(std_fidelities)))")
    println(io, "  Best fidelity: $(Printf.@sprintf("%.6f", maximum(fidelities)))")
    println(io, "  Worst fidelity: $(Printf.@sprintf("%.6f", minimum(fidelities)))")
    println(io, "  Mean convergence time: $(Printf.@sprintf("%.2f", mean(mean_times))) seconds")
end

println("✓ Saved aggregated results")
println("✓ Saved statistics")
println("✓ Saved metadata")
println("✓ Saved trajectory index")
println("✓ Saved summary.txt")
println("\nAll data saved successfully to: $save_path")
println("\nTo load a specific trajectory:")
println("  using JLD2")
println("  @load \"$save_path/trajectories/traj_a1_dda1_seed1.jld2\" traj")

In [ ]:
using JLD2

# Specify the folder path
save_path = "./artifacts/param_sweep_a10_dda25_seeds2_20250930_085535"  # Update with your actual folder name

# Load metadata to get dimensions
@load joinpath(save_path, "metadata.jld2") metadata
n_a = metadata["n_a_points"]
n_dda = metadata["n_dda_points"]
n_seeds = metadata["n_seeds"]

# Load parameter values
@load joinpath(save_path, "results.jld2") a_bounds dda_bounds seed_values

# Initialize array to store all trajectories
all_trajectories = Array{Any}(undef, n_a, n_dda, n_seeds)

# Load each trajectory
println("Loading trajectories...")
for i in 1:n_a
    for j in 1:n_dda
        for seed_idx in 1:n_seeds
            filename = "traj_a$(i)_dda$(j)_seed$(seed_idx).jld2"
            filepath = joinpath(save_path, "trajectories", filename)
            @load filepath traj
            all_trajectories[i, j, seed_idx] = traj
        end
    end
end

println("✓ Loaded $(n_a * n_dda * n_seeds) trajectories")

# Now you can access any trajectory by index
# Example: trajectory for 3rd a_bound, 5th dda_bound, 1st seed
example_traj = all_trajectories[3, 5, 1]

In [ ]:
function var_obj(traj::NamedTrajectory, H_drive::Vector{Matrix{ComplexF64}}, error_op::Matrix{ComplexF64})
    varsys = VariationalQuantumSystem(
        H_drive,
        [error_op]
    )
    T = traj.T
    Δt = traj.Δt[1]
    ww = iso_vec_to_operator(variational_unitary_rollout(traj, varsys)[2][1][:,end])
    d = size(ww)[1]
    return norm(tr(ww'ww)) / (T * Δt)^2 / d
end


In [ ]:

H_drives = [PAULIS.X, PAULIS.Y, PAULIS.Z]

# Initialize storage for var_obj values
var_obj_x = zeros(n_a, n_dda, n_seeds)
var_obj_y = zeros(n_a, n_dda, n_seeds)
var_obj_z = zeros(n_a, n_dda, n_seeds)
var_obj_avg = zeros(n_a, n_dda, n_seeds)

println("\nCalculating var_obj for all trajectories...")

# Calculate var_obj for each trajectory and each Pauli operator
for i in 1:n_a
    for j in 1:n_dda
        for seed_idx in 1:n_seeds
            if !isnothing(all_trajectories[i, j, seed_idx])
                traj = all_trajectories[i, j, seed_idx]
                var_obj_x[i, j, seed_idx] = var_obj(traj, H_drives, PAULIS.X)
                var_obj_y[i, j, seed_idx] = var_obj(traj, H_drives, PAULIS.Y)
                var_obj_z[i, j, seed_idx] = var_obj(traj, H_drives, PAULIS.Z)
                var_obj_avg[i, j, seed_idx] = (var_obj_x[i, j, seed_idx] + 
                                                var_obj_y[i, j, seed_idx] + 
                                                var_obj_z[i, j, seed_idx]) / 3
                
                println("a=$(Printf.@sprintf("%.3e", a_bounds[i])), " *
                        "dda=$(Printf.@sprintf("%.3e", dda_bounds[j])), " *
                        "seed=$seed_idx: " *
                        "var_obj_avg=$(Printf.@sprintf("%.6e", var_obj_avg[i, j, seed_idx]))")
        end
        end
    end
end

# Compute statistics across seeds
mean_var_obj_x = mean(var_obj_x, dims=3)[:, :, 1]
mean_var_obj_y = mean(var_obj_y, dims=3)[:, :, 1]
mean_var_obj_z = mean(var_obj_z, dims=3)[:, :, 1]
mean_var_obj_avg = mean(var_obj_avg, dims=3)[:, :, 1]

std_var_obj_x = std(var_obj_x, dims=3)[:, :, 1]
std_var_obj_y = std(var_obj_y, dims=3)[:, :, 1]
std_var_obj_z = std(var_obj_z, dims=3)[:, :, 1]
std_var_obj_avg = std(var_obj_avg, dims=3)[:, :, 1]

# Create comprehensive visualization
f_var = Figure(size=(2000, 1800))

# Row 1: Mean var_obj for each Pauli and average
ax_x_heat = Axis(f_var[1, 1],
    title="Mean var_obj (X error)",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)
hm_x = Makie.heatmap!(ax_x_heat, log10.(a_bounds), log10.(dda_bounds), mean_var_obj_x, colormap=:viridis)
Colorbar(f_var[1, 2], hm_x, label="Mean var_obj_X")

ax_y_heat = Axis(f_var[1, 3],
    title="Mean var_obj (Y error)",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)
hm_y = Makie.heatmap!(ax_y_heat, log10.(a_bounds), log10.(dda_bounds), mean_var_obj_y, colormap=:viridis)
Colorbar(f_var[1, 4], hm_y, label="Mean var_obj_Y")

ax_z_heat = Axis(f_var[1, 5],
    title="Mean var_obj (Z error)",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)
hm_z = Makie.heatmap!(ax_z_heat, log10.(a_bounds), log10.(dda_bounds), mean_var_obj_z, colormap=:viridis)
Colorbar(f_var[1, 6], hm_z, label="Mean var_obj_Z")

# Row 2: Average var_obj (3D and 2D)
ax_avg_3d = Axis3(f_var[2, 1:2],
    title="Mean Average var_obj (3D)",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)",
    zlabel="var_obj",
    aspect=(1, 1, 0.5)
)

ax_avg_3d.zlabelrotation = π
ax_avg_3d.zreversed = true

x_3d = repeat(log10.(a_bounds), 1, length(dda_bounds))
y_3d = repeat(log10.(dda_bounds)', length(a_bounds), 1)
Makie.surface!(ax_avg_3d, x_3d, y_3d, mean_var_obj_avg, colormap=:plasma, alpha=0.8)
Makie.contour3d!(ax_avg_3d, x_3d, y_3d, mean_var_obj_avg, levels=10, linewidth=1, alpha=0.6)

ax_avg_heat = Axis(f_var[2, 3:4],
    title="Mean Average var_obj",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)
hm_avg = Makie.heatmap!(ax_avg_heat, log10.(a_bounds), log10.(dda_bounds), mean_var_obj_avg, colormap=:plasma)
Colorbar(f_var[2, 5], hm_avg, label="Mean var_obj")

ax_std_heat = Axis(f_var[2, 6],
    title="Std Dev var_obj",
    xlabel="log₁₀(a_bound)",
    ylabel="log₁₀(dda_bound)"
)
hm_std = Makie.heatmap!(ax_std_heat, log10.(a_bounds), log10.(dda_bounds), std_var_obj_avg, colormap=:reds)
Colorbar(f_var[2, 7], hm_std, label="Std Dev")

# Row 3: Cross-sections with seed variability
mid_dda = round(Int, length(dda_bounds)/2)
mid_a = round(Int, length(a_bounds)/2)

colors_seeds = [:red, :blue, :green, :orange, :purple]

ax_cross_a = Axis(f_var[3, 1:3],
    title="var_obj vs a_bound (with seed variability)",
    xlabel="log₁₀(a_bound)",
    ylabel="var_obj"
)

lines!(ax_cross_a, log10.(a_bounds), mean_var_obj_avg[:, mid_dda],
       label="Mean", color=:blue, linewidth=3)
band!(ax_cross_a, log10.(a_bounds),
      mean_var_obj_avg[:, mid_dda] .- std_var_obj_avg[:, mid_dda],
      mean_var_obj_avg[:, mid_dda] .+ std_var_obj_avg[:, mid_dda],
      color=(:blue, 0.3), label="±1 Std")

for seed_idx in 1:n_seeds
    lines!(ax_cross_a, log10.(a_bounds), var_obj_avg[:, mid_dda, seed_idx],
           label="Seed $seed_idx", linewidth=1.5, linestyle=:dash, color=colors_seeds[seed_idx])
end
axislegend(ax_cross_a, position=:rt)

ax_cross_dda = Axis(f_var[3, 4:6],
    title="var_obj vs dda_bound (with seed variability)",
    xlabel="log₁₀(dda_bound)",
    ylabel="var_obj"
)

lines!(ax_cross_dda, log10.(dda_bounds), mean_var_obj_avg[mid_a, :],
       label="Mean", color=:red, linewidth=3)
band!(ax_cross_dda, log10.(dda_bounds),
      mean_var_obj_avg[mid_a, :] .- std_var_obj_avg[mid_a, :],
      mean_var_obj_avg[mid_a, :] .+ std_var_obj_avg[mid_a, :],
      color=(:red, 0.3), label="±1 Std")

for seed_idx in 1:n_seeds
    lines!(ax_cross_dda, log10.(dda_bounds), var_obj_avg[mid_a, :, seed_idx],
           label="Seed $seed_idx", linewidth=1.5, linestyle=:dash, color=colors_seeds[seed_idx])
end
axislegend(ax_cross_dda, position=:lt)

# Row 4: Summary statistics
ax_summary = Axis(f_var[4, 1:6],
    title="var_obj Summary Statistics"
)
hidedecorations!(ax_summary)
hidespines!(ax_summary)

summary_text = """
Variance Objective Statistics (Lower is Better):

Average var_obj (all Pauli errors):
  Mean across all: $(Printf.@sprintf("%.6e", mean(mean_var_obj_avg)))
  Std across params: $(Printf.@sprintf("%.6e", std(mean_var_obj_avg)))
  Avg std across seeds: $(Printf.@sprintf("%.6e", mean(std_var_obj_avg)))
  Min mean: $(Printf.@sprintf("%.6e", minimum(mean_var_obj_avg)))
  Max mean: $(Printf.@sprintf("%.6e", maximum(mean_var_obj_avg)))

Individual Pauli errors:
  X error mean: $(Printf.@sprintf("%.6e", mean(mean_var_obj_x)))
  Y error mean: $(Printf.@sprintf("%.6e", mean(mean_var_obj_y)))
  Z error mean: $(Printf.@sprintf("%.6e", mean(mean_var_obj_z)))

Seed Variability:
  Params with high std (>1e-4): $(sum(std_var_obj_avg .> 1e-4))
  Params with high std (>1e-3): $(sum(std_var_obj_avg .> 1e-3))

Best Parameters (lowest var_obj):
  Min var_obj: $(Printf.@sprintf("%.6e", minimum(mean_var_obj_avg)))
  Location: a_idx=$(argmin(mean_var_obj_avg)[1]), dda_idx=$(argmin(mean_var_obj_avg)[2])
  a_bound: $(Printf.@sprintf("%.3e", a_bounds[argmin(mean_var_obj_avg)[1]]))
  dda_bound: $(Printf.@sprintf("%.3e", dda_bounds[argmin(mean_var_obj_avg)[2]]))
"""

text!(ax_summary, 0.05, 0.5, text=summary_text,
      space=:relative, fontsize=12, font="monospace")

Label(f_var[0, :], "Multi-Seed var_obj Analysis: Parameter Sensitivity",
      fontsize=20, font="bold")

display(f_var)

# Save results
output_file = joinpath(save_path, "var_obj_results.jld2")
@save output_file var_obj_x var_obj_y var_obj_z var_obj_avg mean_var_obj_avg std_var_obj_avg

println("\n✓ var_obj analysis complete!")
println("Results saved to: $output_file")